In [1]:
import os
from dotenv import load_dotenv

from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from datetime import datetime


In [2]:
load_dotenv()
project_endpoint = os.environ["PROJECT_ENDPOINT"]

project_client = AIProjectClient(
    endpoint=project_endpoint,
    # Use Azure Default Credential for authentication
    credential=DefaultAzureCredential(),
)

In [3]:
print('Project Endpoint: ' + os.environ["PROJECT_ENDPOINT"])
print('Azure OpenAI Endpoint: ' + os.environ["AZURE_OPENAI_ENDPOINT"])
print('Azure OpenAI Embedding Deployment Name: ' + os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"])
print('Azure OpenAI Chat Deployment Name: ' + os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"])
print('Azure OpenAI API Version: ' + os.environ["AZURE_OPENAI_API_VERSION"])

print('Azure AI Project Name: ' + os.environ["AZURE_AI_PROJECT_NAME"])

print('Azure OpenAI Resource Group: ' + os.environ["AZURE_OPENAI_RESOURCE_GROUP"])
print('Azure OpenAI Service: ' + os.environ["AZURE_OPENAI_SERVICE"])

print('Azure AI Agent Model Deployment Name: ' + os.environ["AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"])
print('Azure OpenAI Deployment Name: ' + os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"])
#print('AZURE_AI_AGENT_PROJECT_CONNECTION_STRING: ' + os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"])



Project Endpoint: https://servicio-curso-agentes.services.ai.azure.com/api/projects/projecto-curso-agentes
Azure OpenAI Endpoint: https://servicio-curso-agentes.openai.azure.com/
Azure OpenAI Embedding Deployment Name: text-embedding-ada-002
Azure OpenAI Chat Deployment Name: gpt-4o-mini
Azure OpenAI API Version: 2024-12-01-preview
Azure AI Project Name: projecto-curso-agentes
Azure OpenAI Resource Group: rg-cesarisaacdiaz-9109_ai
Azure OpenAI Service: cesarmlgsktp3westeurope
Azure AI Agent Model Deployment Name: gpt-4o-mini
Azure OpenAI Deployment Name: gpt-4o


In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://ai-agents-course-service.openai.azure.com/",
    api_key="21ubRrrUosc26vAsUDlTjeJdEzH8kuTVXIgIfPo7aeR46erEiNR9JQQJ99CBACYeBjFXJ3w3AAAAACOGKkkL",
)

In [ ]:
endpoint = "https://ai-agents-course-service.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"

subscription_key = "21ubRrrUosc26vAsUDlTjeJdEzH8kuTVXIgIfPo7aeR46erEiNR9JQQJ99CBACYeBjFXJ3w3AAAAACOGKkkL"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

In [4]:
from IPython.display import display, HTML, Image
from pathlib import Path


async def run_agent_with_visualization():
    html_output = "<h2>Azure AI Agent Execution</h2>"

    with project_client:
        # Create an instance of the CodeInterpreterTool
        code_interpreter = CodeInterpreterTool()

        # The CodeInterpreterTool needs to be included in creation of the agent
        # Ensure to set the correct model name as deployed in Azure AI Foundry for your use case
        agent = project_client.agents.create_agent(
            model =  "gpt-4o-mini",
            name= "gpt-4o-mini",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
        )

  
        html_output += f"<div><strong>Created agent</strong> with ID: {agent.id}</div>"

        # Create a thread
        thread = project_client.agents.threads.create()
        html_output += f"<div><strong>Created thread</strong> with ID: {thread.id}</div>"

        # User query - display nicely
        user_query = "Could you please create a bar chart for the operating profit using the following data and provide the file to me? Bali: 100 Travelers, Paris: 356 Travelers, London: 900 Travelers, Tokyo: 850 Travellers"
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f5f5f5; border-left:4px solid #007bff; border-radius:4px;'>"
        html_output += "<strong>User:</strong><br>"
        html_output += f"<div style='margin-left:15px'>{user_query}</div>"
        html_output += "</div>"

        # Create a message
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_query,
        )

        # Run the agent - show a "processing" message
        display(HTML(
            html_output + "<div style='color:#007bff'><i>Processing request...</i></div>"))

        # Execute the run
        run = project_client.agents.runs.create_and_process(
            thread_id=thread.id, agent_id=agent.id)

        # Update status
        status_color = 'green' if run.status == 'completed' else 'red'
        html_output += f"<div><strong>Run finished</strong> with status: <span style='color:{status_color}'>{run.status}</span></div>"

        if run.status == "failed":
            html_output += f"<div style='color:red'><strong>Run failed:</strong> {run.last_error}</div>"

        # Get messages from the thread
        messages = project_client.agents.messages.list(thread_id=thread.id)

        # Format assistant response
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f0f7ff; border-left:4px solid #28a745; border-radius:4px;'>"
        html_output += "<strong>Assistant:</strong><br>"

        # Handle messages based on the actual structure
        # First, try to get the assistant's text responses
        try:
            # First approach - if messages is a list of objects with role attribute
            assistant_msgs = [msg for msg in messages if hasattr(
                msg, 'role') and msg.role == "assistant"]

            if assistant_msgs:
                last_msg = assistant_msgs[-1]
                if hasattr(last_msg, 'content'):
                    if isinstance(last_msg.content, list):
                        for content_item in last_msg.content:
                            if hasattr(content_item, 'type') and content_item.type == "text":
                                html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{content_item.text.value}</div>"
                    elif isinstance(last_msg.content, str):
                        html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{last_msg.content}</div>"

            # If no messages were found with the above approach, try a different structure
            if not assistant_msgs:
                # If messages is a class with attributes
                if hasattr(messages, 'data'):
                    for msg in messages.data:
                        if hasattr(msg, 'role') and msg.role == "assistant":
                            if hasattr(msg, 'content'):
                                html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{msg.content}</div>"

        except Exception as e:
            html_output += f"<div style='color:red'><strong>Error processing messages:</strong> {str(e)}</div>"

        html_output += "</div>"

        # Handle image contents based on the actual structure
        saved_images = []
        try:
            # Try to access image_contents as an attribute
            if hasattr(messages, 'image_contents'):
                for image_content in messages.image_contents:
                    file_id = image_content.image_file.file_id
                    file_name = f"{file_id}_image_file.png"
                    project_client.agents.save_file(
                        file_id=file_id, file_name=file_name)
                    saved_images.append(file_name)
                    html_output += f"<div style='margin-top:10px'><strong>Generated Image:</strong> {file_name}</div>"
        except Exception as e:
            html_output += f"<div style='color:orange'><i>Note: No images found or error processing images</i></div>"

        # Handle file path annotations based on the actual structure
        try:
            # Try to access file_path_annotations as an attribute
            if hasattr(messages, 'file_path_annotations'):
                for file_path_annotation in messages.file_path_annotations:
                    file_name = Path(file_path_annotation.text).name
                    project_client.agents.save_file(
                        file_id=file_path_annotation.file_path.file_id, file_name=file_name)
                    html_output += "<div style='margin:10px 0; padding:8px; background-color:#f8f9fa; border:1px solid #ddd; border-radius:4px;'>"
                    html_output += f"<strong>Generated File:</strong> {file_name}<br>"
                    html_output += f"<strong>Type:</strong> {file_path_annotation.type}<br>"
                    html_output += "</div>"
        except Exception as e:
            html_output += f"<div style='color:orange'><i>Note: No file annotations found or error processing files</i></div>"

        # Delete the agent once done
        project_client.agents.delete_agent(agent.id)
        html_output += "<div style='margin-top:10px'><i>Agent deleted after completion</i></div>"

        # Final display of all content
        display(HTML(html_output))

        # Display any saved images
        for img_file in saved_images:
            display(Image(img_file))

# Execute the function
await run_agent_with_visualization()


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.


In [ ]:
import os
import asyncio
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

# Carga las variables (asegúrate de tener python-dotenv instalado y cargar el .env)
# from dotenv import load_dotenv
# load_dotenv()

async def run_agent_clean():
    # 1. Conexión al Proyecto (El "Manager")
    #connection_string = os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"]
    
    project_client = AIProjectClient(
        endpoint='https://servicio-curso-agentes.services.ai.azure.com/api/projects/projecto-curso-agentes',
        #project_connection_string=connection_string, # <--- Se pasa directo como argumento
        credential=DefaultAzureCredential(),
    )

    print(f"Conectando al proyecto...")

    with project_client:
        # 2. Herramienta de Código (Ahora sí funcionará porque tienes Storage)
        code_interpreter = CodeInterpreterTool()

        # 3. Crear el Agente
        # Usamos "gpt-4o" porque es el que TIENES desplegado y PROBADO
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini", 
            name="gpt-4o-mini",
            instructions="Eres un experto ingeniero de datos. Ayuda a visualizar información.",
            tools=code_interpreter.definitions,
        )
        print(f"✅ Agente creado: {agent.id}")

        # 4. Crear Hilo y Mensaje
        thread = project_client.agents.threads.create()
        print(f"✅ Hilo creado: {thread.id}")

        user_query = "Genera un gráfico de barras simple con estos datos: Python 80%, Java 15%, C++ 5%. Guarda la imagen."
        
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_query,
        )
        print(f"📩 Mensaje enviado: {user_query}")

        # 5. Ejecutar (Run)
        print("🏃‍♂️ Ejecutando agente (esto puede tardar unos segundos)...")
        run = project_client.agents.runs.create_and_process(
            thread_id=thread.id, agent_id=agent.id
        )

        print(f"🏁 Estado final: {run.status}")

        if run.status == "completed":
            # Listar mensajes
            messages = project_client.agents.messages.list(thread_id=thread.id)
            for msg in messages.data:
                if msg.role == "assistant":
                    print(f"🤖 Asistente: {msg.content}")
                    # Aquí iría la lógica para descargar los archivos de imagen generados
                    # (Si quieres el código para descargar la imagen, avísame)
        
        # Limpieza (Opcional)
        project_client.agents.delete_agent(agent.id)
        print("🧹 Agente eliminado.")

# Ejecutar
await run_agent_clean() # Si estás en Jupyter
#asyncio.run(run_agent_clean()) # Si es script normal

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

# Tu Connection String
CONN_STRING = os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"]

def inspect_project():
    print("🔍 Iniciando Inspección del Proyecto...")
    
    project_client = AIProjectClient(
        endpoint='https://servicio-curso-agentes.services.ai.azure.com/api/projects/projecto-curso-agentes',
        project_connection_string=CONN_STRING, # <--- Se pasa directo como argumento
        credential=DefaultAzureCredential(),
    )
    
    with project_client:
        # 1. ¿Qué tiene el CodeInterpreter por dentro?
        code_interpreter = CodeInterpreterTool()
        print("\n📦 --- ¿Qué envía el Code Interpreter? ---")
        print(f"Tipo de objeto: {type(code_interpreter)}")
        print(f"Definiciones (JSON): {code_interpreter.definitions}")
        # Verás que solo dice [{'type': 'code_interpreter'}]. 
        # No lleva credenciales, esas las saca del Proyecto.

        # 2. ¿Qué recursos tiene conectados el Proyecto?
        print("\n🔗 --- Lista de Recursos Conectados ---")
        try:
            # Listamos todas las conexiones
            connections = project_client.connections.list()
            
            found_storage = False
            found_openai = False
            
            for conn in connections:
                # Imprimimos nombre y tipo para ver qué tienes
                print(f" - Nombre: {conn.name} | Tipo: {conn.connection_type} | Target: {conn.target}")
                
                if conn.connection_type == "AzureBlob":
                    found_storage = True
                if conn.connection_type == "AzureOpenAI":
                    found_openai = True
            
            print("\n📊 --- Diagnóstico Final ---")
            if found_openai:
                print("✅ OpenAI: CONECTADO (El chat funciona)")
            else:
                print("❌ OpenAI: NO ENCONTRADO (Esto es raro si el chat funcionó)")
                
            if found_storage:
                print("✅ Storage: CONECTADO (El Code Interpreter debería funcionar)")
            else:
                print("❌ Storage: NO ENCONTRADO (¡AQUÍ ESTÁ EL ERROR!)")
                print("   El Code Interpreter necesita un Blob Storage para guardar los archivos.")

        except Exception as e:
            print(f"❌ Error al listar conexiones: {e}")

# Ejecutar
if __name__ == "__main__":
    inspect_project()